# Comparison wind and surge data
This notebook compares the performance of some variants of the sea level rise (SLR) regression model. 
- The first variant includes wind data (wind speed and wind direction) as an explanatory variable. Two versions of the model with wind are considered and compared as the original wind decomposition could be improved upon. 
- The second variant corrects the water levels for the (modelled) storm surges. To improve our understanding of the model we consider the correlation between storm surges and water levels (and also the residuals from the regression model without wind) and we add the storm surge as an explanatory variable to the model.

In [1]:
# Standard Python packages
import io

# Python packages that need to be installed using pip or anaconda:
# For computations
import pandas
import numpy as np

# For GIS related function
import pyproj
from shapely.geometry import LineString, Point

# For loading netcdf
import netCDF4

# For plotting
import matplotlib.pyplot as plt
import bokeh.palettes
import bokeh.plotting
from bokeh.models import HoverTool

# Initialize modules for the jupyter notebook format
from nbformat import v4
%matplotlib inline
bokeh.io.output_notebook()

from IPython.display import display
import shutil

# Disable pandas warnings
pandas.options.mode.chained_assignment = None

Loading BokehJS ...

### Load standard functions

Define function to execute other notebooks.

In [2]:
def execute_notebook(nbfile):
    """Function to run other notebook in this notebook"""
    with io.open(nbfile,encoding="utf8") as f:
        nb = v4.reads_json(f.read())
    
    ip = get_ipython()
    
    for cell in nb.cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.source)


In [3]:
execute_notebook('../satellite/get-data.ipynb')

### Get PSMSL RLR annual data
Annual and monthly sea level data are obtained from PSMSL. 
Wind data and storm surge data are obtained from other sources. See for more details '../satellite/get-data.ipynb'.
Below some data formatting is done.

In [4]:
station_names = [
    'Vlissingen', 
    'Hoek van Holland', 
    'Den Helder', 
    'Delfzijl', 
    'Harlingen', 
    'IJmuiden'
]

# The alpha values are the dominant wind direction for the coast stations.
alphas = {
    "delfzijl": 117,
    "harlingen": 108,
    "den helder": 107,
    "ijmuiden": 110,
    "hoek van holland": 117,
    "vlissingen": 118
}

# Wind station at sea is about 50 km from IJmuiden (Locatie wind data is 50 km uit kust vanuit IJmuiden)
rlr_data = get_station_data(dataset_name='rlr_annual', coastline_code=150, names=station_names, alphas=alphas)
rlr_data

found point 52.3799 3.75
rlr_annual 20 rlr
rlr_annual 22 rlr
rlr_annual 23 rlr
rlr_annual 24 rlr
rlr_annual 25 rlr
rlr_annual 32 rlr


,lat,lon,name,coastline_code,station_code,quality,url,data
id,,,,,,,,
20,51.442222,3.596111,VLISSINGEN,150,101,N,http://www.psmsl.org/data/obtaining/rlr.diagra...,year height interpolated flags ...
22,51.977500,4.120000,HOEK VAN HOLLAND,150,51,N,http://www.psmsl.org/data/obtaining/rlr.diagra...,year height interpolated flags ...
23,52.964444,4.745000,DEN HELDER,150,31,N,http://www.psmsl.org/data/obtaining/rlr.diagra...,year height interpolated flags ...
24,53.326389,6.933056,DELFZIJL,150,1,N,http://www.psmsl.org/data/obtaining/rlr.diagra...,year height interpolated flags ...
25,53.175556,5.409444,HARLINGEN,150,21,N,http://www.psmsl.org/data/obtaining/rlr.diagra...,year height interpolated flags ...
32,52.462222,4.554722,IJMUIDEN,150,41,N,http://www.psmsl.org/data/obtaining/rlr.diagra...,year height interpolated flags ...


The code below also specifies the actions required for creating wind speed components with respect to the angle alpha (dominant wind direction) from the available wind data set. The wind speed is decomposed in two components: main (along the angle) and perp (perpendicular to the coast). In a later stage this option has been cancelled as a method that can be applied globally is preferred.

In [5]:
df = rlr_data.loc[20, 'data']

# Direction is in radials in carthesian system. Calculate nautical direction (in degrees). 
def car2nau(carthesian):
    nautical = ((carthesian * -1) + 90) % 360
    return nautical

dir_car_deg = df['direction'] / np.pi * 180
dir_nau_deg = car2nau(dir_car_deg)

dir_car_deg = df['direction'] / np.pi * 180
dir_nau_deg = car2nau(dir_car_deg)

df['U2main'] = df['speed'] ** 2 * np.cos(np.pi / 180 * (dir_nau_deg - alphas['vlissingen']))
df['U2perp'] = df['speed'] ** 2 * np.sin(np.pi / 180 * (dir_nau_deg - alphas['vlissingen']))

df.tail()

,year,height,interpolated,flags,station,u,v,speed,direction,u2,v2,U2main,U2perp
t,,,,,,,,,,,,,
2013-01-01,2013,-14.0,N,0,20,1.037140,0.346735,2.624925,3.413254,2.228657,3.173827,-4.992611,4.748594
2014-01-01,2014,24.0,N,0,20,1.564820,1.863233,3.526154,2.433411,5.123015,11.433274,-12.135459,-2.707228
2015-01-01,2015,27.0,N,0,20,2.715002,1.360416,3.524224,2.121403,12.157925,6.558920,-10.706787,-6.294839
2016-01-01,2016,30.0,N,0,20,1.751055,0.918265,2.507132,2.084224,4.917603,2.520684,-5.296431,-3.384960
2017-01-01,2017,46.0,N,0,20,2.807551,1.063181,3.324518,1.234811,11.298989,2.050601,-1.681231,-10.923804


In [6]:
rlr_data.loc[20, 'data'].tail(10)

,year,height,interpolated,flags,station,u,v,speed,direction,u2,v2,U2main,U2perp
t,,,,,,,,,,,,,
2008-01-01,2008,19.0,N,0,20,2.186677,1.553167,3.571877,0.906970,8.021732,5.390208,2.223008,-12.563148
2009-01-01,2009,1.0,N,0,20,1.680295,1.128736,2.958833,2.665082,5.088303,5.315205,-8.754041,0.106642
2010-01-01,2010,3.0,N,0,20,0.842527,-0.395506,1.758827,2.393399,1.712336,-1.417380,-2.989897,-0.793781
2011-01-01,2011,7.0,N,0,20,2.143113,1.469787,3.095699,1.707437,9.094103,4.225603,-5.609782,-7.769874
2012-01-01,2012,0.0,N,0,20,2.191390,1.284763,2.981646,1.994099,6.817776,3.249124,-7.029728,-5.442320
2013-01-01,2013,-14.0,N,0,20,1.037140,0.346735,2.624925,3.413254,2.228657,3.173827,-4.992611,4.748594
2014-01-01,2014,24.0,N,0,20,1.564820,1.863233,3.526154,2.433411,5.123015,11.433274,-12.135459,-2.707228
2015-01-01,2015,27.0,N,0,20,2.715002,1.360416,3.524224,2.121403,12.157925,6.558920,-10.706787,-6.294839
2016-01-01,2016,30.0,N,0,20,1.751055,0.918265,2.507132,2.084224,4.917603,2.520684,-5.296431,-3.384960


In [8]:
stations = [20, 22, 23, 24, 25, 32]

grouped = pandas.concat(rlr_data.loc[stations, 'data'].tolist())[['year', 'height', 'u', 'v', 'u2', 'v2', 'U2main', 'U2perp']].groupby(['year'])
mean_df = grouped.mean().reset_index()
# filter out non-trusted part (before NAP)
mean_df = mean_df[mean_df['year'] >= 1890]

station_names = [st.capitalize() for st in rlr_data.loc[stations, 'name'].tolist()]

print(f'The sea water level data from the following stations shall be analyzed: {", ".join(station_names)}')

The sea water level data from the following stations shall be analyzed: Vlissingen, Hoek van holland, Den helder, Delfzijl, Harlingen, Ijmuiden


C:\Users\nicolai.HKV\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until


### Fit models

The following models are fitted:
- linear model without wind or surge: 
    sea water level ~ constant + trend + 18,613 nodal cycle (model without wind)
    
- linear model with wind: 
    sea water level ~ constant + trend + 18,613 nodal cycle + wind components squared (model with wind)
    - decomposition of the wind as is in the sea level monitor (South to North is main, West to East is perp 

- linear model with corrected wind: 
    sea water level ~ constant + trend + 18,613 nodal cycle + corrected wind components squared (model with wind)    
    - improved decomposition of the wind as proposed earlier in Baart et al. (2014) - sea level monitor 2013.

- linear model with surge: 
    (sea water level - storm surge) ~ constant + trend + 18,613 nodal cycle (model with storm surge correction)

The linear models with corrected wind and with surge are specified below. The notebook get-data consists of the definition of the linear model with wind and the model without wind.

First we fit the above models, next we analyze the correlation between sea water levels and storm surges. Finally, we fit another variant of the model with correction for surge: 
    sea water level ~ constant + trend + 18,613 nodal cycle + storm surge (model with storm surge)

In [13]:
def linear_model_with_corrected_wind(df):
    """
    Return the fit from the linear model on the given dataset df.
    Wind and season can be enabled and disabled
    """
    y = df['height']
    X = np.c_[
        df['year']-1970, 
        np.cos(2*np.pi*(df['year']-1970)/18.613),
        np.sin(2*np.pi*(df['year']-1970)/18.613)
    ]
    month = np.mod(df['year'], 1) * 12.0
    names = ['Constant', 'Trend', 'Nodal U', 'Nodal V']
    
    X = np.c_[
        X,
        df['U2main'],
        df['U2perp']
    ]
    names.extend(['Wind U^2 main', 'Wind U^2 perp'])
    
    X = sm.add_constant(X)
    model = sm.OLS(y, X, missing='drop')
    fit = model.fit()
    return fit, names

Fit the four models. 

In [15]:
tables = {}

# first the model without wind
linear_fit, names = linear_model(mean_df, with_wind=False, with_season=False)
tables['without'] = linear_fit.summary(yname='Sea-surface height', xname=names, title='Linear model without wind (1890-current)')
display(tables['without'])

# and then the model with wind
linear_with_wind_fit, names = linear_model(mean_df, with_wind=True, with_season=False)
tables['with wind'] = linear_with_wind_fit.summary(
    yname='Sea-surface height', 
    xname=names,
    title='Linear model with wind (1948-current)'
)
display(tables['with wind'])

# and then the model with wind
linear_with_corrected_wind_fit, names = linear_model_with_corrected_wind(mean_df)
tables['with corrected wind'] = linear_with_corrected_wind_fit.summary(
    yname='Sea-surface height', 
    xname=names,
    title='Linear model with corrected wind (1948-current)'
)
display(tables['with corrected wind'])

# if (linear_fit.aic < linear_with_wind_fit.aic):
#     print('The linear model without wind is a higher quality model (smaller AIC) than the linear model with wind.')
# else:
#     print('The linear model with wind is a higher quality model (smaller AIC) than the linear model without wind.')

# Load surge and convert from meters to mm
# File surge.pkl (~ 100 MB) is saved in a local local folder)
# surge = pandas.read_pickle('d:/Documents/2695.50 Zeespiegelstijging 2018/Data/surge.pkl') * 1000
# surge = pandas.read_pickle('d:/Users/Documents/2695.50 Zeespiegelstijging 2018/Data/surge.pkl') * 1000
surge = pandas.read_pickle('surge.pkl') * 1000

# Calculate average over given stations, per year
average = surge[station_names].groupby(surge.index.year).mean().mean(axis=1)

# Construct dataframe and add to mean_df
surge_per_year = pandas.DataFrame(data=[average.mean()] * len(mean_df), index=mean_df['year'], columns=['surge'])
surge_per_year.loc[average.index, 'surge'] = average.values
surge_per_year.index.name = 'year'
if 'surge' not in mean_df.columns:
    mean_df = mean_df.merge(surge_per_year.reset_index(), on='year')

mean_df.head()

mean_df_corrected = mean_df.copy()
mean_df_corrected['height'] -= mean_df_corrected['surge']

linear_fit_corrected, names = linear_model(mean_df_corrected, with_wind=False, with_season=False)
table = linear_fit_corrected.summary(
    yname='Sea-surface height', 
    xname=names,
    title='Linear model corrected for surge (1979-current)'
)
display(table)

print(f'Linear model without wind or surge (1890-current): AIC = {linear_fit.aic}')
print(f'Linear model with wind (1948-current): AIC = {linear_with_wind_fit.aic}')
print(f'Linear model with corrected wind (1948-current): AIC = {linear_with_corrected_wind_fit.aic}')
print(f'Linear model corrected for surge (1979-current): AIC = {linear_fit_corrected.aic}')

<class 'statsmodels.iolib.summary.Summary'>
"""
                   Linear model without wind (1890-current)                   
==============================================================================
Dep. Variable:     Sea-surface height   R-squared:                       0.855
Model:                          GLSAR   Adj. R-squared:                  0.852
Method:                 Least Squares   F-statistic:                     275.8
Date:                Thu, 29 Nov 2018   Prob (F-statistic):           2.04e-54
Time:                        19:13:09   Log-Likelihood:                -606.92
No. Observations:                 127   AIC:                             1222.
Df Residuals:                     123   BIC:                             1233.
Df Model:                           3                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Constant     -23.9339      2.913     -8.217      0.000     -29.643     -18.225
Trend          1.9153      0.067     28.655      0.000       1.784       2.046
Nodal U        4.9877      3.704      1.347      0.178      -2.271      12.247
Nodal V      -12.7148      3.518     -3.614      0.000     -19.611      -5.819
==============================================================================
Omnibus:                        3.685   Durbin-Watson:                   1.681
Prob(Omnibus):                  0.158   Jarque-Bera (JB):                3.347
Skew:                          -0.395   Prob(JB):                        0.188
Kurtosis:                       3.084   Cond. No.                         57.3
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Linear model with wind (1948-current)                     
==============================================================================
Dep. Variable:     Sea-surface height   R-squared:                       0.909
Model:                          GLSAR   Adj. R-squared:                  0.905
Method:                 Least Squares   F-statistic:                     354.1
Date:                Thu, 29 Nov 2018   Prob (F-statistic):           1.90e-70
Time:                        19:13:09   Log-Likelihood:                -577.70
No. Observations:                 127   AIC:                             1167.
Df Residuals:                     121   BIC:                             1184.
Df Model:                           5                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Constant     -70.7314      5.365    -13.184      0.000     -81.247     -60.216
Trend          1.8636      0.055     33.627      0.000       1.755       1.972
Nodal U        3.5117      2.859      1.228      0.219      -2.092       9.115
Nodal V      -12.0050      2.882     -4.166      0.000     -17.654      -6.356
Wind $u^2$     7.3026      0.731      9.986      0.000       5.869       8.736
Wind $v^2$     0.0290      1.092      0.027      0.979      -2.112       2.170
==============================================================================
Omnibus:                        2.182   Durbin-Watson:                   1.532
Prob(Omnibus):                  0.336   Jarque-Bera (JB):                1.735
Skew:                          -0.152   Prob(JB):                        0.420
Kurtosis:                       3.485   Cond. No.                         120.
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

C:\Users\nicolai.HKV\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:153: RuntimeWarning: invalid value encountered in reduce
  umr_maximum(a, axis, None, out, keepdims),
C:\Users\nicolai.HKV\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:154: RuntimeWarning: invalid value encountered in reduce
  umr_minimum(a, axis, None, None, keepdims),


<class 'statsmodels.iolib.summary.Summary'>
"""
               Linear model with corrected wind (1948-current)                
==============================================================================
Dep. Variable:     Sea-surface height   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.701
Method:                 Least Squares   F-statistic:                     33.30
Date:                Thu, 29 Nov 2018   Prob (F-statistic):           1.40e-16
Time:                        19:13:09   Log-Likelihood:                -331.58
No. Observations:                  70   AIC:                             675.2
Df Residuals:                      64   BIC:                             688.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Constant        -40.3364      7.268     -5.550      0.000     -54.856     -25.817
Trend             1.9705      0.178     11.079      0.000       1.615       2.326
Nodal U           2.9813      4.900      0.608      0.545      -6.808      12.771
Nodal V         -17.0889      5.043     -3.388      0.001     -27.164      -7.013
Wind U^2 main    -2.0190      0.964     -2.093      0.040      -3.946      -0.092
Wind U^2 perp    -1.6213      0.647     -2.505      0.015      -2.914      -0.328
==============================================================================
Omnibus:                        1.454   Durbin-Watson:                   1.807
Prob(Omnibus):                  0.483   Jarque-Bera (JB):                1.069
Skew:                          -0.301   Prob(JB):                        0.586
Kurtosis:                       3.069   Cond. No.                         51.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
               Linear model corrected for surge (1979-current)                
==============================================================================
Dep. Variable:     Sea-surface height   R-squared:                       0.886
Model:                          GLSAR   Adj. R-squared:                  0.883
Method:                 Least Squares   F-statistic:                     394.1
Date:                Thu, 29 Nov 2018   Prob (F-statistic):           6.98e-63
Time:                        19:13:11   Log-Likelihood:                -589.52
No. Observations:                 127   AIC:                             1187.
Df Residuals:                     123   BIC:                             1198.
Df Model:                           3                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Constant     -22.9576      2.267    -10.128      0.000     -27.400     -18.515
Trend          1.9175      0.056     34.159      0.000       1.808       2.028
Nodal U        4.2873      3.112      1.378      0.168      -1.812      10.386
Nodal V      -11.0740      3.205     -3.455      0.001     -17.356      -4.792
==============================================================================
Omnibus:                        3.220   Durbin-Watson:                   1.510
Prob(Omnibus):                  0.200   Jarque-Bera (JB):                2.694
Skew:                          -0.270   Prob(JB):                        0.260
Kurtosis:                       3.467   Cond. No.                         57.3
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

Linear model without wind or surge (1890-current): AIC = 1221.8375172201627
Linear model with wind (1948-current): AIC = 1167.4049778949807
Linear model with corrected wind (1948-current): AIC = 675.1573830599831
Linear model corrected for surge (1979-current): AIC = 1187.0395629455115


The first three linear models are fitted to the same sea level data and so we may compare the AIC values. In the fourth model  the annual average sea level is corrected for the storm surge (model outcomes are subtracted from the sea levels). Hence, we cannot compare the AIC values of this model with the other models.

Note that the surge data are available from 1979 onwards and the wind data are available from 1948 onwards. For the years where storm surge or wind data are missing mean values are used in the linear regressions.

The lower the AIC, the better the model performance. 
- The linear model with wind (the current sea level monitor SLR model) outperforms the model without wind on the same dataset.
- The linear model with the corrected wind performs even better than the model with the 'original' wind.

In the linear model corrected for surge the annual average sea level is corrected for the storm surge from storm surge model outcomes. This approach implicitly assumes the model surge output to be match perfectly with sea water levels and hence can be subtracted from the sea level. We shall assess the correlation between surge and sea water levels below. 

Plots of the storm surges from the global tidal surge reanalysis (GTSR) model can be found in 
'\Read modelled storm surges.ipynb'. Statistical analyses of the sea levels and storm surges can be found in '\Statistical tests.ipynb'.

In [11]:
print(f'Average surge for ({", ".join(station_names)}) = {surge[station_names].mean().mean()} mm')

NameError: name 'surge' is not defined

In [12]:
# plot the model with wind. 
fig = bokeh.plotting.figure(x_range=(1860, 2020), plot_width=900, plot_height=400)

fig.circle(
    mean_df.year,
    mean_df.height,
    line_width=1,
    legend='Annual mean sea level (data)',
    color='black',
    alpha=0.5
)

fig.line(
    linear_fit.model.exog[:, 1] + 1970, 
    linear_fit.predict(), 
    line_width=3, 
    legend='Current sea level (linear model without wind)', 
    color='green',
    alpha=0.5
)

fig.line(
    linear_with_wind_fit.model.exog[:, 1] + 1970, 
    linear_with_wind_fit.predict(), 
    line_width=3, 
    alpha=0.5,
    #color='blue',
    legend='Current sea level (linear model with wind)'
)

fig.line(
    linear_with_corrected_wind_fit.model.exog[:, 1] + 1970, 
    linear_with_corrected_wind_fit.predict(), 
    line_width=3, 
    alpha=0.5,
    legend='Current sea level (linear model with corrected wind)',
    color='purple'
)


fig.circle(
    mean_df_corrected.year[mean_df_corrected.year >= 1979],
    mean_df_corrected.height[mean_df_corrected.year >= 1979],
    line_width=1,
    legend='Annual mean sea level corrected for storm surge (data)',
    color='orange',
    alpha=0.5
)

fig.line(
    linear_fit_corrected.model.exog[:, 1] + 1970, 
    linear_fit_corrected.predict(), 
    line_width=3, 
    legend='Current sea level minus storm surge (linear model)', 
    color='red',
    line_dash='dashed',
    alpha=0.5
)

fig.legend.location = "top_left"
fig.yaxis.axis_label = 'waterlevel [mm] above NAP'
fig.xaxis.axis_label = 'year'
#fig.legend.click_policy = "hide"

bokeh.io.show(fig)

NameError: name 'linear_with_corrected_wind_fit' is not defined

The graph shows the annual sea levels and the fit of the linear model (without and with wind). Also, the graph shows the annual sea level minus the storm surge and the corresponding fit of the linear model.
Both the wind as well as the storm surge seem to explain the yearly variation in the sea level quite well. 

We now add the modelled surge as an exploratory variable to the linear model (in stead of the wind) and estimate the regression coefficient. One would expect a coefficient close to 1 (hypothesis). 
N.B. The years prior to 1979 are disregarded.

Let us now first determine the correlation between the storm surge and the residuals of the regression model without wind.

In [ ]:
yearmin = 1979
indices = mean_df['year'] >= yearmin
#print(indices)

fig, ax = plt.subplots()

xdata = mean_df.loc[indices, 'surge'].values
#print(mean_df.loc[indices, 'surge'])
#print(xdata, len(xdata))

# @ GUUS: deze regel werkt bij mij niet naar behoren
#ydata = linear_fit.resid.values[indices] 
ydata = linear_fit.resid.tail(len(xdata)).values

#print(linear_fit.resid)
#print(linear_fit.resid.values)
#print(linear_fit.resid.tail(len(xdata)).values)
#ydata = linear_fit.resid.tail(len(xdata)).values
#print(ydata, len(ydata))

ax.scatter(xdata, ydata, s=10)
ax.grid()
ax.set_ylabel('Residuen [mm]')
ax.set_xlabel('Surge [mm]')
ax.set_aspect(1.0)

x = np.linspace(-100, 100, 100)
p = np.polyfit(xdata, ydata, 1)
y = np.polyval(p, x)
ax.plot(x, y, color='k')

ax.set_title(np.corrcoef(xdata, ydata)[0, 1])
print(np.square(np.corrcoef(xdata, ydata)[0, 1]))

The correlation coefficient is about 0.86 and so the storm surge explains about 73% of the variation in the residuals.

Next we compare the annual sea level to the annual storm surge.

In [ ]:
yearmin = 1979
indices = mean_df['year'] >= yearmin

fig, ax = plt.subplots()

ax.scatter(mean_df.loc[indices, 'surge'].values, mean_df.loc[indices, 'height'].values, s=10)
ax.grid()
ax.set_xlabel('Surge [mm]')
ax.set_ylabel('Height [mm]')
ax.set_aspect(1.0)

x = np.linspace(-100, 100, 100)
p = np.polyfit(mean_df.loc[indices, 'surge'].values, mean_df.loc[indices, 'height'].values, 1)
y = np.polyval(p, x)
ax.plot(x, y, color='k')

ax.set_title(np.corrcoef(mean_df.loc[indices, 'surge'].values, mean_df.loc[indices, 'height'].values)[0, 1])
print(np.square(np.corrcoef(mean_df.loc[indices, 'surge'].values, mean_df.loc[indices, 'height'].values)[0, 1]))

Again, we find that a large part of the variation in the annual sea level can be explained by the yearly variation in storm surge. Hence, we define a linear model with the (modelled) storm surge as an explanatory variable. That is, 

sea water level ~ constant + trend + 18,613 nodal cycle + storm surge (model with storm surge)

In [ ]:
def linear_model_with_surge(df):
    """
    Return the fit from the linear model on the given dataset df.
    Wind and season can be enabled and disabled
    """
    y = df['height']
    X = np.c_[
        df['year']-1970, 
        np.cos(2*np.pi*(df['year']-1970)/18.613),
        np.sin(2*np.pi*(df['year']-1970)/18.613),
        df['surge']# * (df['year'] >= 1979)    
    ]
    month = np.mod(df['year'], 1) * 12.0
    names = ['Constant', 'Trend', 'Nodal U', 'Nodal V', 'Surge']
        
    X = sm.add_constant(X)
    model = sm.OLS(y, X, missing='drop')
    fit = model.fit()
    return fit, names

In [ ]:
linear_fit_with_surge, names = linear_model_with_surge(mean_df)
tables['with surge'] = linear_fit_with_surge.summary(
    yname='Sea-surface height', 
    xname=names, 
    title='Linear model with surge (1979-current)'
)

In [ ]:
display(tables['without'])
display(tables['with wind'])
display(tables['with corrected wind'])
display(tables['with surge'])

print(f'Linear model without wind or surge (1890-current): AIC = {linear_fit.aic}')
print(f'Linear model with wind (1948-current): AIC = {linear_with_wind_fit.aic}')
print(f'Linear model with corrected wind (1948-current): AIC = {linear_with_corrected_wind_fit.aic}')
print(f'Linear model with surge (1979-current): AIC = {linear_fit_with_surge.aic}')

The coefficient of the surge (0.9009) is close to 1. The null hypothesis that is equals 1 cannot be rejected at a significance level of 5%. This means that we can just subtract the storm surge values from the sea water levels.

We can now compare the AIC values of the models with wind and with surge. This comparison is not really fair, because the model with surge uses far more 'average' values than the models with wind. This might be the explanation that the model with corrected wind performs better than the model with surge.

In [ ]:
# plot the models again 
fig = bokeh.plotting.figure(x_range=(1860, 2020), plot_width=900, plot_height=400)

fig.circle(
    mean_df.year,
    mean_df.height,
    line_width=1,
    legend='Annual mean sea level (data)',
    color='black',
    alpha=0.5
)

fig.line(
    linear_fit.model.exog[:, 1] + 1970, 
    linear_fit.predict(), 
    line_width=3, 
    legend='Current sea level (linear model without wind)', 
    color='green',
    line_dash='dashed',
    alpha=0.5
)

fig.line(
    linear_with_wind_fit.model.exog[:, 1] + 1970, 
    linear_with_wind_fit.predict(), 
    line_width=3, 
    alpha=0.5,
    legend='Current sea level (linear model with wind)'
)

fig.line(
    linear_with_corrected_wind_fit.model.exog[:, 1] + 1970, 
    linear_with_corrected_wind_fit.predict(), 
    line_width=3, 
    alpha=0.5,
    legend='Current sea level (linear model with corrected wind)',
    color='purple'
)

fig.line(
    linear_fit_with_surge.model.exog[:, 1] + 1970, 
    linear_fit_with_surge.predict(), 
    line_width=3, 
    legend='Current sea level (linear model with surge)', 
    color='red',
    alpha=0.5
)

#fig.circle(
#    mean_df_corrected.year[mean_df_corrected.year >= 1979],
#    mean_df_corrected.height[mean_df_corrected.year >= 1979],
#    line_width=1,
#    legend='Annual sea level corrected for storm surge (data)',
#    color='orange',
#    alpha=0.5
#)

fig.legend.location = "top_left"
fig.yaxis.axis_label = 'waterlevel [mm] above NAP'
fig.xaxis.axis_label = 'year'
#fig.legend.click_policy = "hide"

bokeh.io.show(fig)

The plot shows that the model with (the corrected) wind and the model with storm surge correction both perform quite well in the period 1979-now. As shown before the AIC-value of the model with surge is lower than the AIC-value of the model with wind. The (only) d8fference9between these models is that the number of wind observations is higher than the number of storm surge observations. Note agaibefore the year  me9n values have been used to fill up the wind and storm surge variables before the year 1948 and 1979, respectively. 

To make a fair comparison we define a wind data set with mean values in the same years as the storm surge data set (before the year 1979) and perform a regression on this new wind data set. Next we compare the AIC-value again with that of the model with surge.

In [ ]:
df1979 = mean_df.copy()

for col in ['u2', 'v2', 'u2main', 'u2perp']:
    df1979.loc[df1979['year'] < 1979, col] = df1979.at[0, col]
    
display(df1979.loc[80:100])

In [ ]:
linear_with_corrected_wind_fit, names = linear_model_with_corrected_wind(df1979)
table = linear_with_corrected_wind_fit.summary(
    yname='Sea-surface height', 
    xname=names,
    title='Linear model with corrected wind (1979-current)'
)
display(table)

linear_fit_with_surge, names = linear_model_with_surge(mean_df)
table = linear_fit_with_surge.summary(
    yname='Sea-surface height', 
    xname=names, 
    title='Linear model with surge (1979-current)'
)
display(table)

print(f'Linear model with corrected wind (1979-current): AIC = {linear_with_corrected_wind_fit.aic}')
print(f'Linear model with surge (1979-current): AIC = {linear_fit_with_surge.aic}')

The AIC-value of the model with surge is lower, suggesting that the fit of this model is better. 

Finally, let us show the plots of the regression lines.

In [ ]:
# plot the models again 
fig = bokeh.plotting.figure(x_range=(1860, 2020), plot_width=900, plot_height=400)

fig.circle(
    mean_df.year,
    mean_df.height,
    line_width=1,
    legend='Annual mean sea level (data)',
    color='black',
    alpha=0.5
)

fig.line(
    linear_with_corrected_wind_fit.model.exog[:, 1] + 1970, 
    linear_with_corrected_wind_fit.predict(), 
    line_width=3, 
    alpha=0.5,
    legend='Current sea level (linear model with corrected wind)',
    color='blue'
)

fig.line(
    linear_fit_with_surge.model.exog[:, 1] + 1970, 
    linear_fit_with_surge.predict(), 
    line_width=3, 
    legend='Current sea level (linear model with surge)', 
    color='red',
    alpha=0.5
)

fig.legend.location = "top_left"
fig.yaxis.axis_label = 'waterlevel [mm] above NAP'
fig.xaxis.axis_label = 'year'
#fig.legend.click_policy = "hide"

bokeh.io.show(fig)